In [2]:
import json
import requests
import pandas as pd
import numpy as np

headers = {"Authorization": "KakaoAK 1b1096bed1659c02314973cadeb6e7df"}
geocoding_url = 'https://dapi.kakao.com/v2/local/search/address.json?query='

def kakao_geocoder(address):
    try:
        url = geocoding_url + address
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        return pd.Series({'위도': float(match_first['y']), '경도': float(match_first['x'])})

    except:
        return None  



In [4]:
raw = pd.read_csv('./data/도매_EDA_완료_new_v3.0.csv')
raw2 = pd.read_csv('./data/지역_추출_도매_데이터_new_v3.0.csv')

In [5]:
def change_columns_name(df):
    df.rename(columns = {'DELNG_DE' : '경락일',
                            'PBLMNG_WHSAL_MRKT_NM' : '공영_도매시장명',
                            'PBLMNG_WHSAL_MRKT_CD' : '공영_도매시장_코드',
                            'CPR_NM' : '법인명',
                            'CPR_CD' : '법인코드',
                            'RISENO' : '원표번호',
                            'BIDTIME' : '경매시간',
                            'AUC_SE_NM' : '경매구분',
                            'AUC_SE_CODE' : '경매구분코드',
                            'ORGNO' : '일련번호',
                            'PRDLST_NM' : '품목명',
                            'PRDLST_CD' : '품목코드',
                            'SPCIES_NM' : '품종명',
                            'SPCIES_CD' : '품종코드',
                            'CPR_USE_PRDLST_NM' : '법인사용품명',
                            'CPR_USE_SPCIES_NM' : '법인사용품종명',
                            'PRICE' : '가격',
                            'DELNGBUNDLE_QY' : '거래단량',
                            'STNDRD' : '규격',
                            'STNDRD_CD' : '규격코드',
                            'GRAD' : '등급',
                            'GRAD_CD' : '등급코드',
                            'SHIPMNT_SE_CODE' : '출하구분코드',
                            'SHIPMNT_SE_NM' : '출하구분명',
                            'SANJI_CD' : '산지코드',
                            'SANJI_NM' : '산지명',
                            'CPR_USE_SANJI_CD' : '법인산지코드',
                            'CPR_USE_SANJI_NM' : '법인산지명',
                            'DELNG_QY' : '거래량(원본)',
                            'VOLUME' : '거래량'
                            }, inplace = True)
change_columns_name(raw)
change_columns_name(raw2)
df = raw.copy()
df2 = raw2.copy()

In [6]:
sanji = df['산지명'].str.split(' ') # 문자열을 split 메소드로 분리
df['산지_시'] = sanji.str.get(0)

df['산지_시'] = df['산지_시'].map({'충북':'충청북도',
                             '경기':'경기도',
                             '충남':'충청남도',
                             '전북':'전라북도',
                             '전남':'전라남도',
                             '경북':'경상북도',
                             '경남':'경상남도',
                             '부산':'부산광역시',
                             '울산':'울산광역시',
                             '서울':'서울특별시',
                             '강원':'강원도',
                             '대구':'대구광역시',
                             '인천':'인천광역시',
                             '광주':'광주광역시',
                             '대전':'대전광역시',
                             '제주도':'제주특별자치도',
                             '경기/시흥시':'경기도',
                             '경기/파주시':'경기도',
                             '전북전주시':'전라북도',
                             '세종':'세종특별자치시',
                             '충남/아산시/신창면':'충청남도',
                             '경남밀양시삼랑진읍':'경상남도',
                             '경기양주시':'경기도',
                             '해외':np.NaN,
                             '스위스':np.NaN,
                             '뉴질랜드':np.NaN,
                             })



df['시장_구'] = df['공영_도매시장명'].map({'청주도매시장':'충청북도 청주시',
                             '충주도매시장':'충청북도 충주시',
                             '천안도매시장':'충청남도 천안시',
                             '전주도매시장':'전라북도 전주시',
                             '익산도매시장':'전라북도 익산시',
                             '정읍도매시장':'전라북도 정읍시',
                             '목포농산시장':'전라남도 목포시',
                             '순천도매시장':'전라남도 순천시',
                             '구미도매시장':'경상북도 구미시',
                             '창원팔용도매시장':'경상남도 창원시',
                             '울산도매시장':'울산광역시 남구',
                             '진주도매시장':'경상남도 진주시',
                             '서울가락도매':'서울특별시 송파구',
                             '서울강서도매':'서울특별시 강서구',
                             '부산엄궁도매':'부산광역시 사상시',
                             '부산반여도매':'부산광역시 해운대구',
                             '대구북부도매':'대구광역시 북구', # 북구 
                             '인천남촌도매':'인천광역시 남동구',
                             '인천삼산도매':'인천광역시 부평구',
                             '광주각화도매':'광주광역시 북구', # 북구
                             '광주서부도매':'광주광역시 서구', # 서구
                             '대전오정도매':'대전광역시 대덕구',
                             '대전노은도매':'대전광역시 유성구',
                             '수원도매시장':'경기도 수원시',
                             '안양도매시장':'경기도 안양시',
                             '안산도매시장':'경기도 안산시',
                             '구리도매시장':'경기도 구리시',
                             '춘천도매시장':'강원도 춘천시',
                             '원주도매시장':'강원도 원주시',
                             '강릉도매시장':'강원도 강릉시',
                             '포항도매시장':'경상북도 포항시',
                             '여수농산시장':np.NaN, # 정확한 주소 나오지 않아서 제외
                             '안동도매시장' : '경상북도 안동시'
                             })
df = df.dropna()

In [7]:
# 거래량(톤) 추가하고 연도,월 추가
df['거래량(톤)'] = df['거래량'] / 1000
df['경락일'] = pd.to_datetime(df['경락일'], format="%Y%m%d")
df['연도'] = df['경락일'].dt.year
df['월'] = df['경락일'].dt.month
df_2020 = df[df['연도']!=2021]

# [도매시장] 연도별, 월별, 최근 한달 도매시장별로 거래량 합계, 평균 가격 
df_year_market = (df[["시장_구","공영_도매시장명","연도", "거래량(톤)","가격"]].groupby(["시장_구","공영_도매시장명","연도"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_month_market = (df_2020[["시장_구","공영_도매시장명","월", "거래량(톤)","가격"]].groupby(["시장_구","공영_도매시장명","월"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_last = df.set_index("경락일").last("1M").reset_index() # 최근 한달과 거래량 
df_last_market = (df_last[["시장_구","공영_도매시장명","경락일", "거래량(톤)","가격"]].groupby(["시장_구","공영_도매시장명","경락일"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()

# 
df_year_market[['위도','경도']] = df_year_market.apply(lambda x: kakao_geocoder(x['시장_구']), axis=1)
df_month_market[['위도','경도']] = df_month_market.apply(lambda x: kakao_geocoder(x['시장_구']), axis=1)
df_last_market[['위도','경도']] = df_last_market.apply(lambda x: kakao_geocoder(x['시장_구']), axis=1)

# 
df_year_market['거래량(톤)'] = np.around(df_year_market['거래량(톤)'])
df_month_market['거래량(톤)'] = np.around(df_month_market['거래량(톤)'])
df_last_market['거래량(톤)'] = np.around(df_last_market['거래량(톤)'])


# [산지] 연도별, 월별, 최근 한달 산지별로 거래량 합계, 평균 가격 
df_year_sanji = (df[["산지_시","연도", "거래량(톤)","가격"]].groupby(["산지_시","연도"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_month_sanji = (df_2020[["산지_시","월", "거래량(톤)","가격"]].groupby(["산지_시","월"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_last = df.set_index("경락일").last("1M").reset_index() # 최근 한달과 거래량 
df_last_sanji = (df_last[["산지_시","경락일", "거래량(톤)","가격"]].groupby(["산지_시","경락일"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()

# 
df_year_sanji[['위도','경도']] = df_year_sanji.apply(lambda x: kakao_geocoder(x['산지_시']), axis=1)
df_month_sanji[['위도','경도']] = df_month_sanji.apply(lambda x: kakao_geocoder(x['산지_시']), axis=1)
df_last_sanji[['위도','경도']] = df_last_sanji.apply(lambda x: kakao_geocoder(x['산지_시']), axis=1)

# 
df_year_sanji['거래량(톤)'] = np.around(df_year_market['거래량(톤)'])
df_month_sanji['거래량(톤)'] = np.around(df_month_market['거래량(톤)'])
df_last_sanji['거래량(톤)'] = np.around(df_last_market['거래량(톤)'])

#
df_year_sanji['가격'] = np.around(df_year_market['가격'])
df_month_sanji['가격'] = np.around(df_month_market['가격'])
df_last_sanji['가격'] = np.around(df_last_market['가격'])


In [8]:
change_columns_name(raw2)
df2 = raw2.copy()
df2['거래량(톤)'] = df2['거래량'] / 1000
df2['경락일'] = pd.to_datetime(df2['경락일'], format="%Y%m%d")
df2['연도'] = df2['경락일'].dt.year
df2['월'] = df2['경락일'].dt.month
df2_2020 = df2[df2['연도']!=2021]


df_year_main_sanji = (df2[["산지명","연도", "거래량(톤)","가격"]].groupby(["산지명","연도"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_month_main_sanji = (df2_2020[["산지명","월", "거래량(톤)","가격"]].groupby(["산지명","월"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df2_last = df2.set_index("경락일").last("1M").reset_index() # 최근 한달 거래량 
df_last_main_sanji = (df2_last[["산지명","경락일", "거래량(톤)","가격"]].groupby(["산지명","경락일"])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()


In [9]:
import plotly.express as px

In [11]:
############ [도매시장 연도별] 거래량 지도 시각화 #################################

market_geo_year = px.scatter_mapbox(df_year_market,
                    lat='위도',
                    lon='경도',
                    color='거래량(톤)',
                    #color='공영_도매시장명'
                    size='거래량(톤)',
                    hover_name='공영_도매시장명',
                    animation_frame='연도',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    #color_continuous_scale=px.colors.qualitative.Pastel,
                    size_max=50,
                    zoom=6,
                    width=600, 
                    height=600,
                    animation_group='공영_도매시장명',
                    title='도매시장별 거래량'
                    )
market_geo_year.update_layout(
    font_family="AppleGothic",
    )
market_geo_year.update_layout(mapbox={'style':'light'},mapbox_accesstoken='pk.eyJ1IjoieWprMDYxNCIsImEiOiJja3IxYjZ4ZzYwdm40MnJyeHJpNGlnOGZ3In0.OCm6t8C0TfXw7rzfZMkYMg',showlegend=True)
market_geo_year.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
market_geo_year["layout"].pop("updatemenus")# PLAY 버튼 disabled
market_geo_year.show()
#market_geo_year = plot(market_geo_year, output_type='div')

In [14]:

##################[도매시장 최근 한달] 거래량 지도 시각화 ########################## 

#df_last_market['경락일'] = df_last_market['경락일'].dt.strftime('%Y-%m-%d')

market_geo_1month = px.scatter_mapbox(df_last_market,
                    lat='위도',
                    lon='경도',
                    color='거래량(톤)',
                    #color='공영_도매시장명'
                    size='거래량(톤)',
                    hover_name='공영_도매시장명',
                    animation_frame='경락일',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    #color_continuous_scale=px.colors.qualitative.Pastel,
                    size_max=50,
                    zoom=6,
                    width=600, 
                    height=600,
                    animation_group='공영_도매시장명',
                    title='도매시장별 거래량'
                    )
market_geo_1month.update_layout(
    font_family="AppleGothic",
    )
#'open-street-map'    
market_geo_1month.update_layout(mapbox={'style':'light'},mapbox_accesstoken='pk.eyJ1IjoieWprMDYxNCIsImEiOiJja3IxYjZ4ZzYwdm40MnJyeHJpNGlnOGZ3In0.OCm6t8C0TfXw7rzfZMkYMg',showlegend=True)
market_geo_1month.update_layout(margin={'r':0,'t':0,'l':0,'b':0})

market_geo_1month["layout"].pop("updatemenus")
market_geo_1month.show()
#market_geo_1month = plot(market_geo_1month, output_type='div')



market_bar_1month = px.bar(df_last_market, x= '공영_도매시장명',
                  y ='거래량(톤)',
              animation_frame="경락일",
              color='공영_도매시장명',
              color_discrete_sequence=px.colors.qualitative.Pastel1)
market_bar_1month.update_layout(
    font_family="AppleGothic",
    )
market_bar_1month.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)",
                  font=dict(
        size=15,
        color="black"
    ),
xaxis=dict(title='경락일',linewidth=1,linecolor = 'gray',gridcolor='rgba(0,0,0,0)'),
yaxis=dict(title='거래량(톤)',showline=True,linewidth=1, linecolor = 'gray',gridcolor='gray'))
market_bar_1month["layout"].pop("updatemenus")
market_bar_1month.show()
#market_bar_1month = plot(market_bar_1month, output_type='div')


In [17]:
###############[도매시장 가격] TOP5#######################################################

df_last_market_top5 = df_last_market[(df_last_market['공영_도매시장명'] == '서울가락도매')|(df_last_market['공영_도매시장명'] == '구리도매시장')|
(df_last_market['공영_도매시장명'] == '부산엄궁도매')|(df_last_market['공영_도매시장명'] == '서울강서도매')|(df_last_market['공영_도매시장명'] == '부산반여도매')]
df_last_market_top5 = df_last_market_top5[df_last_market_top5['경락일'] != '2021-08-07']
market_bar_top5 = px.bar(df_last_market_top5,
            x = '공영_도매시장명',
            y = '가격',
            color='공영_도매시장명',
            color_discrete_sequence=px.colors.qualitative.Pastel1,
            animation_frame='경락일')
market_bar_top5.update_layout(
    font_family="AppleGothic",
    )
market_bar_top5.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)",
                  font=dict(
        size=15,
        color="black"
    ),
xaxis=dict(range = [df_last_market_top5['경락일'].min(),df_last_market_top5['경락일'].max()],title='경락일',linewidth=1,linecolor = 'gray',gridcolor='rgba(0,0,0,0)'),
# top5 도매 xaxis 값 변경 (0, max(df_last_market_top5['가격'].max()))
yaxis=dict(range = [0,np.around(max(df_last_market_top5['가격']))],title='가격',showline=True,linewidth=1, linecolor = 'gray',gridcolor='gray'))

market_bar_top5.show()
#market_bar_top5 = plot(market_bar_top5, output_type='div')

In [18]:
##################[도매시장] top5 도매시장 가격 변화 추이 ###############

df_day_market_top5 = (df[['경락일','공영_도매시장명','거래량(톤)','가격']].groupby(['경락일','공영_도매시장명'])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()
df_day_market_top5 = df_day_market_top5[(df_day_market_top5['공영_도매시장명'] == '서울가락도매')|(df_day_market_top5['공영_도매시장명'] == '구리도매시장')|
(df_day_market_top5['공영_도매시장명'] == '부산엄궁도매')|(df_day_market_top5['공영_도매시장명'] == '서울강서도매')|(df_day_market_top5['공영_도매시장명'] == '부산반여도매')]

markettop5_price_line = px.line(df_day_market_top5, x='경락일', y='가격', color='공영_도매시장명',color_discrete_sequence=px.colors.qualitative.Pastel1)
#markettop5_price_line.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)")
markettop5_price_line.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)")
markettop5_price_line.update_layout(
    font_family="AppleGothic",
    )
markettop5_price_line.update_yaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='gray')
markettop5_price_line.update_xaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='rgba(0,0,0,0)')
markettop5_price_line.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            #dict(count=1, label="1개월 전", step="month", stepmode="backward"),
            dict(count=1, label="최근 1달", step="month", stepmode="todate"),
            dict(count=6, label="6개월전", step="month", stepmode="backward"),
            dict(count=1, label="1년전", step="year", stepmode="backward"),
            dict(label='전체기간',step="all")
        ])
    )
)
markettop5_price_line.show()
#markettop5_price_line = plot(markettop5_price_line, output_type='div')


In [20]:
#################[산지] 월별 거래량 지도 시각화 #######################

sanji_geo_month = px.scatter_mapbox(df_month_sanji,
                    lat='위도',
                    lon='경도',
                    color='거래량(톤)',
                    #color='공영_도매시장명'
                    size='거래량(톤)',
                    hover_name='산지_시',
                    animation_frame='월',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    #color_continuous_scale=px.colors.qualitative.Pastel,
                    size_max=50,
                    zoom=6,
                    width=600, 
                    height=600,
                    animation_group='산지_시',
                    )
sanji_geo_month.update_layout(
    font_family="AppleGothic",
    )
sanji_geo_month.update_layout(mapbox={'style':'light'},mapbox_accesstoken='pk.eyJ1IjoieWprMDYxNCIsImEiOiJja3IxYjZ4ZzYwdm40MnJyeHJpNGlnOGZ3In0.OCm6t8C0TfXw7rzfZMkYMg',showlegend=True)
sanji_geo_month.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
#fig["layout"].pop("updatemenus") PLAY 버튼 disabled
sanji_geo_month.show()
#sanji_geo_month = plot(sanji_geo_month, output_type='div')

In [23]:
#####################[산지] 월별 가격 시각화 #######################################

sanji_geo_month_price = px.scatter_mapbox(df_month_sanji,
                    lat='위도',
                    lon='경도',
                    color='가격',
                    #color='공영_도매시장명'
                    size='가격',
                    hover_name='산지_시',
                    animation_frame='월',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    #color_continuous_scale=px.colors.qualitative.Pastel,
                    size_max=50,
                    zoom=6,
                    width=600, 
                    height=600,
                    animation_group='산지_시',
                    )
sanji_geo_month_price.update_layout(
    font_family="AppleGothic",
    )
sanji_geo_month_price.update_layout(mapbox={'style':'light'},mapbox_accesstoken='pk.eyJ1IjoieWprMDYxNCIsImEiOiJja3IxYjZ4ZzYwdm40MnJyeHJpNGlnOGZ3In0.OCm6t8C0TfXw7rzfZMkYMg',showlegend=True)
sanji_geo_month_price.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
#fig["layout"].pop("updatemenus") PLAY 버튼 disabled
sanji_geo_month_price.show()
#sanji_geo_month_price = plot(sanji_geo_month_price, output_type='div')



In [24]:
######################[주요 산지] 거래량 시각화######################

df_day_main_sanji = (df2[['경락일','산지명','거래량(톤)','가격']].groupby(['경락일','산지명'])).agg({'거래량(톤)':'sum', '가격':'mean'}).reset_index()

sanji_volumne_line = px.line(df_day_main_sanji, x='경락일', y='거래량(톤)', color='산지명',color_discrete_sequence=px.colors.qualitative.Pastel1)
sanji_volumne_line.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)")
sanji_volumne_line.update_layout(
    font_family="AppleGothic",
    )
sanji_volumne_line.update_yaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='gray')
sanji_volumne_line.update_xaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='rgba(0,0,0,0)')
sanji_volumne_line.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            #dict(count=1, label="1개월 전", step="month", stepmode="backward"),
            dict(count=1, label="최근 1달", step="month", stepmode="todate"),
            dict(count=6, label="6개월전", step="month", stepmode="backward"),
            dict(count=1, label="1년전", step="year", stepmode="backward"),
            dict(label='전체기간',step="all")
        ])
    )
)
sanji_volumne_line.show()
#sanji_volumne_line = plot(sanji_volumne_line, output_type='div')

In [25]:
######################[주요 산지] 거래량 시각화######################

sanji_price_line = px.line(df_day_main_sanji, x='경락일', y='가격', color='산지명',color_discrete_sequence=px.colors.qualitative.Pastel1)
sanji_price_line.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)")
sanji_price_line.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)")
sanji_price_line.update_layout(
    font_family="AppleGothic",
    )
sanji_price_line.update_yaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='gray')
sanji_price_line.update_xaxes(showline=True, linewidth=1, linecolor = 'gray',gridcolor='rgba(0,0,0,0)')
sanji_price_line.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            #dict(count=1, label="1개월 전", step="month", stepmode="backward"),
            dict(count=1, label="최근 1달", step="month", stepmode="todate"),
            dict(count=6, label="6개월전", step="month", stepmode="backward"),
            dict(count=1, label="1년전", step="year", stepmode="backward"),
            dict(label='전체기간',step="all")
        ])
    )
)
sanji_price_line.show()
#sanji_price_line = plot(sanji_price_line, output_type='div')

In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

######################[주요 산지] 거래량+ 가격 시각화######################

sanji_month_volume_price = make_subplots(rows=1, cols=2,
                    horizontal_spacing= 0.15,
                   subplot_titles=('산지별 거래량','산지별 가격'))

sanji_month_volume_price.add_trace(go.Bar(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='강원도(평창+)']['거래량(톤)'],
                 #mode='lines', 
                 name="강원도", 
                 marker=dict(color='lightcoral')), row=1, col=1)
sanji_month_volume_price.add_trace(go.Bar(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='경기도(구리+)']['거래량(톤)'],
                 #mode='lines', 
                 name="경기도", 
                 marker=dict(color='lightcoral')), row=1, col=1)
sanji_month_volume_price.add_trace(go.Bar(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='경상남도(부산+)']['거래량(톤)'],
                 #mode='lines', 
                 name="경상남도", 
                 marker=dict(color='skyblue')), row=1, col=1)
sanji_month_volume_price.add_trace(go.Bar(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='전라남도(진도+)']['거래량(톤)'],
                 #mode='lines', 
                 name="전라남도", 
                 marker=dict(color='skyblue')), row=1, col=1)


sanji_month_volume_price.add_trace(go.Scatter(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='강원도(평창+)']['가격'],
                 mode='lines+markers', 
                 name="강원도", 
                #  opacity=0.5,
                 line=dict(width=3,dash='solid',color='lightcoral'),
                 marker=dict(color='lightcoral')), row=1, col=2)
sanji_month_volume_price.add_trace(go.Scatter(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='경기도(구리+)']['가격'],
                 mode='lines+markers', 
                 name="경기도", 
                 line=dict(width=3,dash='longdash',color='firebrick'),
                 marker=dict(color='firebrick')), row=1, col=2)
sanji_month_volume_price.add_trace(go.Scatter(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='경상남도(부산+)']['가격'],
                 mode='lines+markers', 
                 name="경상남도",
                #  opacity=0.5,
                 line=dict(width=3,dash='solid',color='skyblue'), 
                 marker=dict(color='skyblue')), row=1, col=2)
sanji_month_volume_price.add_trace(go.Scatter(x=df_month_main_sanji['월'], y=df_month_main_sanji[df_month_main_sanji['산지명']=='전라남도(진도+)']['가격'],
                 mode='lines+markers', 
                 name="전라남도", 
                 line=dict(width=3,dash='longdash',color='cornflowerblue'),
                 marker=dict(color='cornflowerblue')), row=1, col=2)

# fig.update_layout(title='<b>주요 산지별 거래량과 가격</b>', showlegend=True)
sanji_month_volume_price.update_layout(barmode='stack')

sanji_month_volume_price.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor="rgba(0,0,0,0)",
                  font=dict(
        size=15,
        color="black"
    ),
    title={
        #'text': '주요 산지별 거래량과 가격',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},)

sanji_month_volume_price.update_xaxes(title_text="월", row=1, col=1)
sanji_month_volume_price.update_xaxes(title_text="월", row=1, col=2)
sanji_month_volume_price.update_yaxes(title_text="거래량(톤)", row=1, col=1)
sanji_month_volume_price.update_yaxes(title_text="가격", row=1, col=2)
sanji_month_volume_price.update_layout(
    font_family="AppleGothic",
    )

for i in sanji_month_volume_price['layout']['annotations']:
    i['font'] = dict(size=18)

sanji_month_volume_price.show()
#sanji_month_volume_price = plot(sanji_month_volume_price, output_type='div')
